In [64]:
import pandas as pd
import numpy as np

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [65]:
# Import genetic distance matrix IDs
with open('../epidemic_simulation_data/genetic_distance_matrices/genetic_distance_matrix_names.txt', 'r') as file:
    matrix_ids = file.read().splitlines()

# Import pairwise genetic distance matrix
pandas2ri.activate()

## Load the RData file
ro.r['load']('../epidemic_simulation_data/genetic_distance_matrices/genetic_distance_matrix.RData')

## Access genetic distance matrix
gdist_seqs_dm = ro.r['gdist_seqs_dm']

## Check if it's a dist object and convert it to a matrix in R
if isinstance(gdist_seqs_dm, ro.Vector):
    gdist_seqs_dm_matrix = ro.r.as_matrix(gdist_seqs_dm)

    ### Convert the R matrix to a NumPy array
    gendist_matrix = np.array(gdist_seqs_dm_matrix)
else:
    ### Directly convert to a NumPy array if it's already in an appropriate format
    gendist_matrix = np.array(gdist_seqs_dm)

## Confirm object creation
print(gendist_matrix)


[[0.         0.00080725 0.00279541 ... 0.0070593  0.00716106 0.00549977]
 [0.00080725 0.         0.00198602 ... 0.0062452  0.00634686 0.0046874 ]
 [0.00279541 0.00198602 0.         ... 0.00553359 0.00563517 0.00397732]
 ...
 [0.0070593  0.0062452  0.00553359 ... 0.         0.00010086 0.00168274]
 [0.00716106 0.00634686 0.00563517 ... 0.00010086 0.         0.00178381]
 [0.00549977 0.0046874  0.00397732 ... 0.00168274 0.00178381 0.        ]]


In [66]:
# Import and process data frames
sampling_pool_df = pd.read_csv('../epidemic_simulation_data/sampling_pool.csv')
domestic_pool_df = pd.read_csv('../epidemic_simulation_data/domestic_pool.csv')

## Merge the data frames on 'id' to combine all features and labels for each observation
combined_df = pd.merge(sampling_pool_df, domestic_pool_df, on=['id', 'date', 'collection_location', 'new_cases', 'mobility', 'sequencing_intensity'], how='outer')

## Replace NaN values in 'travel_history' column
combined_df['travel_history'] = combined_df['travel_history'].fillna('international')

## Convert 'date' to a numerical feature, e.g., days since the first date in the dataset
combined_df['date'] = pd.to_datetime(combined_df['date'])
combined_df['date_numeric'] = (combined_df['date'] - combined_df['date'].min()).dt.days


## Rearrange by 'id'
### Use pd.Categorical to impose the order of 'matrix_ids' on the combined DataFrame
combined_df['id_ordered'] = pd.Categorical(
    combined_df['id'], categories=matrix_ids, ordered=True)

### Sort the combined DataFrame by this new 'id_ordered' column
combined_df = combined_df.sort_values('id_ordered')

### Drop the temporary 'id_ordered' column
combined_df = combined_df.drop(columns=['id_ordered'])

print(combined_df)

               id       date collection_location  new_cases  mobility  \
7187        seq_0 2020-10-13            domestic          1     -0.01   
7189       seq_37 2020-08-28            domestic         19     -0.01   
7213      seq_661 2020-07-23            domestic        184     -0.01   
89      seq_10057 2020-07-11         intl_source        515      0.01   
3941    seq_41495 2020-05-03         intl_source       8888      0.01   
...           ...        ...                 ...        ...       ...   
1261   seq_126100 2020-05-28         intl_source       5271      0.01   
4273    seq_48219 2020-04-30         intl_source       8844      0.01   
10071  seq_144133 2020-05-18            domestic       9947     -0.01   
2009   seq_142575 2020-05-19         intl_source       6847      0.01   
11168  seq_160664 2020-05-12            domestic      12229     -0.01   

       sequencing_intensity travel_history  date_numeric  
7187               1.000000      no_travel           273  
7189 

In [73]:
# Feature engineering, step A - prepare features from data frames
## Features include 'new_cases', 'sequencing_intensity', 'mobility', 'date_numeric',
## and genetic distances.

## Prepare features and labels
features = combined_df[['new_cases', 'sequencing_intensity', 'mobility', 'date_numeric']].values

## Add distance features here as needed, depending on the structure of your distance matrix
labels = combined_df['travel_history']  ### Assuming this column contains X, Y, and possibly A labels

## Convert labels to a numeric format for machine learning: 0 for international, 1 for travel, and
## 2 for no_travel (unlabeled)
numeric_labels = labels.map({'international': 0, 'travel': 1, 'no_travel': 2}).values

In [74]:
# Feature engineering, step B - incorporate genetic distances as features
## Approach 1: we'll use the distance to the nearest reference observation as a feature
min_distance_to_reference = np.min(gendist_matrix, axis=1)  ### Minimum distance for each observation

## Incorporate this distance feature into your existing features
features = np.hstack([features, min_distance_to_reference.reshape(-1, 1)])


In [75]:
# AL model setup and training - minimum genetic distances
## Initial split: separate labeled (international samples and samples with travel history) from unlabeled
## (no travel history) data
labeled_indices = np.where(numeric_labels != 2)[0]
unlabeled_indices = np.where(numeric_labels == 2)[0]

# You would use 'features_with_distance' in place of 'features' for training and predictions
X_labeled_with_distance = features[labeled_indices]
X_unlabeled_with_distance = features[unlabeled_indices]

## Second split: separate labeled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_labeled_with_distance, y_labeled, test_size=0.2, random_state=42)

## Train initial model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate initial model
y_pred = model.predict(X_test)
print(f"Initial accuracy: {accuracy_score(y_test, y_pred)}")

Initial accuracy: 0.9225260416666666


In [76]:
# Active Learning Loop conditions
n_iterations = 50  ### Define the number of iterations for the active learning loop

def query_for_label(index):
    ## Placeholder for your label querying mechanism
    ## This function should return the true label for the sample identified by `index`
    return np.random.randint(0, 2)  ### Example: simulate obtaining a label


In [77]:
# Active Learning Loop - minimum genetic distances
for iteration in range(n_iterations):
    X_unlabeled = features[unlabeled_indices]
    probs = model.predict_proba(X_unlabeled_with_distance) ### Use minimum genetic distance as feature
    
    uncertainty = np.max(probs, axis=1)
    query_idx = np.argmax(uncertainty)
    
    true_label = query_for_label(unlabeled_indices[query_idx])
    
    X_train = np.vstack([X_train, X_unlabeled_with_distance[query_idx]]) ### Use minimum genetic distance as feature
    y_train = np.append(y_train, true_label)
    
    unlabeled_indices = np.delete(unlabeled_indices, query_idx)
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    print(f"Iteration {iteration + 1}, accuracy: {accuracy_score(y_test, y_pred)}")

Iteration 1, accuracy: 0.9225260416666666
Iteration 2, accuracy: 0.9225260416666666
Iteration 3, accuracy: 0.9225260416666666
Iteration 4, accuracy: 0.9225260416666666
Iteration 5, accuracy: 0.9225260416666666
Iteration 6, accuracy: 0.9225260416666666
Iteration 7, accuracy: 0.9225260416666666
Iteration 8, accuracy: 0.9225260416666666
Iteration 9, accuracy: 0.9225260416666666
Iteration 10, accuracy: 0.9225260416666666
Iteration 11, accuracy: 0.9225260416666666
Iteration 12, accuracy: 0.9225260416666666
Iteration 13, accuracy: 0.9225260416666666
Iteration 14, accuracy: 0.9225260416666666
Iteration 15, accuracy: 0.9225260416666666
Iteration 16, accuracy: 0.9225260416666666
Iteration 17, accuracy: 0.9225260416666666
Iteration 18, accuracy: 0.9225260416666666
Iteration 19, accuracy: 0.9225260416666666
Iteration 20, accuracy: 0.9225260416666666
Iteration 21, accuracy: 0.9225260416666666
Iteration 22, accuracy: 0.9225260416666666
Iteration 23, accuracy: 0.9225260416666666
Iteration 24, accura

In [78]:
# Predict labels of unlabeled (no travel history) sequences

## Extract features for unlabeled data
X_unlabeled = features[unlabeled_indices]

## Use the trained model to predict the labels of the unlabeled data
predicted_labels = model.predict(X_unlabeled)

## This task is a binary classification exercise which identifies unlabeled sequences as either
## 'linked to the source' or 'not linked to the source'.
## The model will output 0 (unlinked) or 1 (linked).

## For illustration, let's map 0 to 'A' and 1 to 'X' (though in your case, 'Y' predictions remain 'Y')
predicted_labels_mapped = np.where(predicted_labels == 0, 'unlinked', 'linked')

## Attach these predictions back to your original dataset
ids_unlabeled = combined_df.loc[unlabeled_indices, 'id'].values

predicted_df = pd.DataFrame({
    'id': ids_unlabeled,
    'predicted_label': predicted_labels_mapped
})

print(predicted_df)

             id predicted_label
0        seq_16        unlinked
1        seq_60        unlinked
2       seq_144        unlinked
3       seq_228        unlinked
4       seq_263        unlinked
...         ...             ...
8333   seq_9961        unlinked
8334  seq_99716        unlinked
8335  seq_99785        unlinked
8336  seq_99841        unlinked
8337  seq_99876        unlinked

[8338 rows x 2 columns]


In [79]:
# Summarise predicted labels
## Count the number of linked and unlinked observations in the predicted labels
linked_count = (predicted_df['predicted_label'] == 'linked').sum()
unlinked_count = (predicted_df['predicted_label'] == 'unlinked').sum()

print(f"Number of linked observations (predicted as 'with travel history'): {linked_count}")
print(f"Number of unlinked observations (remaining as 'without travel history'): {unlinked_count}")


Number of linked observations (predicted as 'X'): 177
Number of unlinked observations (remaining as 'Y'): 8161
